In [1]:
import numpy as np
import os
import torch
import cv2
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import  segmentation_models_pytorch  as smp
%matplotlib inline

/home/lisijiang/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


定义路径和名称

In [18]:
gt_nms = ['22.png', '182.png', '270.png', '203.png', '1254.png', '410.png']
gt_dir = r'/boot/data1/kang_data/zkxt21_seaice/trainData/gt'
src_dir = r'/boot/data1/kang_data/zkxt21_seaice/trainData/image'
model_dict = {
    'unet': '../submit/compared_methods/UNet/20240110_124331',
    'pspnet': '../submit/compared_methods/pspnet/20240110_192717',
    'fpn': '../submit/compared_methods/fpn/20240110_192850',
    'linknet': '../submit/compared_methods/Linknet/20240110_124514',
    'deeplabv3': '../submit/compared_methods/deeplabv3/20240110_125054',
    'deeplabv3+': '../submit/compared_methods/deeplabv3+/20240110_125433',
    'EffUNet_stage4': '../submit/EffUNet_stage4/20240109_194957'
}
### https://htmlcolorcodes.com/

海冰图片

In [21]:
for gt_nm in gt_nms:
    print(gt_nm)
    src = Image.open(os.path.join(src_dir, gt_nm.split('.')[0]+'.tif')).convert('RGB')
    src.save('/code/lisijiang/zkxt_src/submit/论文图片/模型预测/src/'+gt_nm)
    

22.png
182.png
270.png
203.png
1254.png
410.png


gt

In [ ]:
for gt_nm in gt_nms:
    print(gt_nm)
    src = Image.open(os.path.join(gt_dir, gt_nm))
    src.save('/code/lisijiang/zkxt_src/submit/论文图片/模型预测/gt/'+gt_nm)
    

### UNet

In [4]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['unet'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/unet/{gt_nm}')


22.png
182.png
270.png
203.png
1254.png
410.png


### PSPNet

In [5]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['pspnet'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/pspnet/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png


### FPN

In [6]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['fpn'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/fpn/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png


Linknet

In [7]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['linknet'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/linknet/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png


Deeplabv3

In [8]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['deeplabv3'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/deeplabv3/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png


Deeplabv3+

In [9]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['deeplabv3+'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/deeplabv3+/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png


EffUNet_stage4

In [12]:
for gt_nm in gt_nms:
    print(gt_nm)
    pred_msk = np.asarray(Image.open(os.path.join(model_dict['EffUNet_stage4'], 'test_predict', gt_nm))).copy()
    gt_msk = np.asarray(Image.open(os.path.join(gt_dir, gt_nm))).copy()

    pred_msk[pred_msk>0] = 1
    gt_msk[gt_msk>0] = 1

    img_msk = np.zeros((512,512,3))
    # assign true positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 1)] = 208
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 1)] = 211
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 1)] = 212
    # assign false positive
    img_msk[:, :, 0][np.logical_and(pred_msk == 1, gt_msk == 0)] = 203
    img_msk[:, :, 1][np.logical_and(pred_msk == 1, gt_msk == 0)] = 67
    img_msk[:, :, 2][np.logical_and(pred_msk == 1, gt_msk == 0)] = 53
    # assign false negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 1)] = 46
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 1)] = 134
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 1)] = 193
    # assign True negative
    img_msk[:, :, 0][np.logical_and(pred_msk == 0, gt_msk == 0)] = 39
    img_msk[:, :, 1][np.logical_and(pred_msk == 0, gt_msk == 0)] = 55
    img_msk[:, :, 2][np.logical_and(pred_msk == 0, gt_msk == 0)] = 70

    # gt_nm_wo_ext = gt_nm.split('.')[0]
    # print(img_msk.shape)
    Image.fromarray(img_msk.astype(np.uint8)).save(f'/code/lisijiang/zkxt_src/submit/论文图片/模型预测/EffUNet_stage4/{gt_nm}')

22.png
182.png
270.png
203.png
1254.png
410.png
